In [24]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast

In [54]:
df = pd.read_csv("output/features_labeled_v2.csv")

In [55]:
df['abnormalpattern_noerror_ranked'] = df['abnormalpattern_noerror_ranked'].apply(lambda x: ast.literal_eval(x))

In [56]:
df['abnormalpattern_error_ranked'] = df['abnormalpattern_error_ranked'].apply(lambda x: ast.literal_eval(x))

In [57]:
df['inject_type'].unique()

array(['cpu_contention', 'return', 'cpu_consumed', 'exception',
       'network_delay'], dtype=object)

In [58]:
log_template_miner = get_miner()

In [59]:
def get_pattern_prompt(col):
    """
    Generate a prompt for the LLM to predict the root cause type based on the abnormal pattern information.
    """
    patterns_details = []
    for pattern in col:
        event_details = []
        event_details.append(
            f"Actual Event Trace: ({from_id_to_template(int(pattern['events'].split('_')[0]), log_template_miner)}) -> "
            f"({from_id_to_template(int(pattern['events'].split('_')[1]), log_template_miner)})"
        )
        event_details.append(f"Abnormal Score: {pattern['score']}")
        event_details.append(f"Depth: {pattern['deepth']}")
        event_details.append(f"Pod: {pattern['pod']}")
        if "events_expected" in pattern:
            event_details.append(
                f"Expected Event Trace: {from_id_to_template(int(pattern['events_expected'].split('_')[0]), log_template_miner)} -> {from_id_to_template(int(pattern['events_expected'].split('_')[1]), log_template_miner)}"
            )
        patterns_details.append("\n".join(event_details))

    prompt = "\n\n".join(patterns_details)
    return prompt

In [60]:
df['abnormalpattern_noerror_ranked_prompt'] = df['abnormalpattern_noerror_ranked'].apply(get_pattern_prompt)
df['abnormalpattern_error_ranked_prompt'] = df['abnormalpattern_error_ranked'].apply(get_pattern_prompt)

In [61]:
pd.set_option('display.max_colwidth', None)
df.head()

,abnormal_time,abnormalpattern_noerror_ranked,abnormalpattern_error_ranked,inject_time,inject_pod,inject_type,abnormalpattern_noerror_ranked_prompt,abnormalpattern_error_ranked_prompt
0,2022-08-22 03:54,"[{'events': '5_12', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_67', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_184', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_6', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_8', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_10', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_85', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_14', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_77', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_101', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}]",[],2022-08-22 03:53:54,frontend-579b9bff58-t2dbm,cpu_contention,Actual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.ProductCatalogService/GetProduct start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CurrencyService/Convert start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (CpuUsageRate(%) Service: alarm)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CurrencyService/GetSupportedCurrencies start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CartService/GetCart start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.RecommendationService/ListRecommendations start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.AdService/GetAds start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.ShippingService/GetQuote start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CartService/AddItem start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.ProductCatalogService/ListProducts start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm,
1,2022-08-22 03:55,"[{'events': '5_12', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_67', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_184', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_6', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_8', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_10', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_85', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_14', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '5_77'

In [64]:
df.columns

Index(['abnormal_time', 'abnormalpattern_noerror_ranked',
       'abnormalpattern_error_ranked', 'inject_time', 'inject_pod',
       'inject_type', 'abnormalpattern_noerror_ranked_prompt',
       'abnormalpattern_error_ranked_prompt'],
      dtype='object')

In [65]:
df[['abnormal_time', 'abnormalpattern_noerror_ranked_prompt', 'abnormalpattern_error_ranked_prompt', 'inject_time', 'inject_pod',
       'inject_type']].to_csv("output/features_labeled_4llm_v1.csv", index=False)